# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [110]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [111]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [112]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [113]:
print(len(full_data_rows_list))
print(full_data_rows_list[0])

8056
['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91']


In [114]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [115]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
    

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [116]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events
    WITH REPLICATION = {'class':'SimpleStrategy',
                        'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [117]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [118]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


In [119]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
task_one_df = df.loc[(df['sessionId']==338) & (df['itemInSession']==4)][['artist','song','length']]
task_one_data = task_one_df.values.tolist()
db_one = """CREATE TABLE IF NOT EXISTS task_one(artist TEXT, song TEXT, length FLOAT, 
               PRIMARY KEY(artist, song, length))
"""
try:
    session.execute(db_one)
except Exception as e:
    print(e)
#import data into database one
import_one = """INSERT INTO task_one(artist, song, length)
                VALUES (%s, %s, %s)
"""
for each in task_one_data:
    session.execute(import_one,(each[0],each[1],each[2]))


#### Do a SELECT to verify that the data have been inserted into each table

In [120]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

try:
    validate_one = session.execute('SELECT * FROM task_one;')
except Exception as e:
    print(e)
    
for each in validate_one:
    print(each.artist,',',each.song,',',each.length)

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


In [121]:
#I use my own code instead of prepared code.

# # We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
# file = 'event_datafile_new.csv'

# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
# ## TO-DO: Assign the INSERT statements into the `query` variable
#         query = "<ENTER INSERT STATEMENT HERE>"
#         query = query + "<ASSIGN VALUES HERE>"
#         ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
#         ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         session.execute(query, (line[#], line[#]))

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [123]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
task_two_df = df.loc[(df['userId']==10) | (df['sessionId']==182)].sort_values(by='itemInSession')[['artist','song','firstName','lastName']]
task_two_data = task_two_df.values.tolist()
db_two = """CREATE TABLE IF NOT EXISTS task_two(artist TEXT, song TEXT, firstName TEXT, lastName TEXT, 
               PRIMARY KEY(artist, song))
"""
try:
    session.execute(db_two)
except Exception as e:
    print(e)


import_two = """INSERT INTO task_two(artist, song, firstName, lastName)
                VALUES (%s, %s, %s, %s)
"""
for each in task_two_data:
    try:
        session.execute(import_two,(each[0],each[1],each[2],each[3]))
    except Exception as e:
        print(e)


                    

In [124]:
try:
    validate_two = session.execute('SELECT * FROM task_two;')
except Exception as e:
    print(e)
    
for each in validate_two:
    print(each.artist,',', each.song,',', each.firstname,each.lastname)

Wu-Tang Clan , Impossible , Sylvie Cruz
Vader , Carnal , Sylvie Cruz
Nightwish , The Islander , Sylvie Cruz
Percubaba , Intro , Sylvie Cruz
Chromeo , Momma's Boy , Sylvie Cruz
KC And The Sunshine Band , Get Down Tonight (Miami Mix) , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
I Set My Friends On Fire , But The NUNS Are Watching , Sylvie Cruz
Pavement , Mercy:The Laundromat , Sylvie Cruz
Ricardo Arjona , Como Duele (Album) , Sylvie Cruz
Molotov , Marciano (I Turned Into A Martian) , Sylvie Cruz
Klaus Badelt , Moonlight Serenade , Sylvie Cruz
Amy Winehouse , You Know I'm No Good , Sylvie Cruz
Watain , Sworn To The Dark , Sylvie Cruz
The White Stripes , Seven Nation Army (Album Version) , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Rilo Kiley , A Man/Me/Then Jim (Album Version) , Sylvie Cruz
Matt Redman , Blessed Be Your Name , Sylvie Cruz
Samael , God's Snake , Sylvie Cruz
Cameo , Word Up! , Sylvie Cruz
Black Eyed Peas , Pump It , Sylvie Cruz
Eminem , Criminal , Syl

In [125]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
task_three_df = df.loc[df['song']=='All Hands Against His Own'].sort_values(by='itemInSession')[['firstName','lastName']]
task_three_data = task_three_df.values.tolist()
db_three = """CREATE TABLE IF NOT EXISTS task_three(firstName TEXT, lastName TEXT, 
               PRIMARY KEY(firstName, lastName))
"""
try:
    session.execute(db_three)
except Exception as e:
    print(e)

import_three = """INSERT INTO task_three(firstName, lastName)
                VALUES (%s, %s)
"""
for each in task_three_data:
    try:
        session.execute(import_three,(each[0],each[1]))
    except Exception as e:
        print(e)
                    

In [126]:
try:
    validate_two = session.execute('SELECT * FROM task_three;')
except Exception as e:
    print(e)
    
for each in validate_two:
    print(each.firstname,each.lastname)

Tegan Levine
Jacqueline Lynch
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [108]:
tables = ['task_one','task_two','task_three']
for each in tables:
    rows = session.execute('DROP TABLE IF EXISTS ' + str(each) + ';')

### Close the session and cluster connection¶

In [109]:
session.shutdown()
cluster.shutdown()